In [1]:
import pandas as pd
import numpy as np

Extracting basic information sheet information first

In [2]:
info = pd.read_excel('K11 Employment Particulars and Roaster (Cleaned).xlsx', 'Basic Information')

info.fillna(value="NA", inplace=True)

def dummy(charvalue):
    y = charvalue.replace("'", "")
    return y
info['Highest_Qualification'] = info['Highest_Qualification'].apply(dummy)

In [3]:
output_file = open("data extracted.sql","a")

for index, row in info.iterrows():
    # Employee basic information table
    to_write = "insert into Emp_Basic_Information (First_Name, Last_Name, Gender, Marital_Status, Date_Of_Birth, Nationality, Religion, Race, Blood_Group, Place_Of_Birth, Identification_Type, Identification_No, Pass_Type, Highest_Qualification, Mobile_No, Email, CREATED_BY) values ('" + row['First_Name'] + "', '" + row['Last_Name'] + "', '" + row['Gender'] + "', '" + row['Marital_Status'] + "', '" + str(row['Date_Of_Birth']) + "', '" + row['Nationality'] + "', '" + row['Religion'] + "', '" + row['Race'] + "', '" + row['Blood_Group'] + "', '" + row['Place_Of_Birth'] + "', '" + row['Identification_Type'] + "', '" + str(row['Identification_No']) + "', '" + row['Pass_Type'] + "', '" + row['Highest_Qualification'] + "', '" + str(row['Mobile_No']) + "', '" + row['Email'] + "' , 1);\n"
    output_file.write(to_write)
    
    # Login Access table
    to_write = "insert into Login_Access (Employee_ID, Username, Password_Hashed, CREATED_BY) values ('" + str(row['ID']) + "', '" + str(row['Identification_No']) + "', \"" + "$2y$10$QuV6azSrw48E26.EZGjqOuj1jF0sIuk/fALR8qmATGvVEJr4H9CxW" + "\" , 1);\n"
    output_file.write(to_write)
    
    # Telegram table
    to_write = "insert into Telegram (Employee_ID, Telegram_ID, Chat_ID, CREATED_BY) values ('" + str(row['ID']) + "', " + "Null" + ", " + "Null" + " , 1);\n"
    output_file.write(to_write)
    
    # Employee employment details table
    to_write = "insert into Emp_Employment_Details (Employee_ID, Joining_Date, Employment_Type, Designation, Department, CREATED_BY) values ('" + str(row['ID']) + "', '" + str(row['Date Joined']) + "', '" + row['Full Time or Part Time'] + "', '" + "Security Officer" + "', '" + "NA" + "' , 1);\n"
    output_file.write(to_write)
    
    # Employee emergency contact table
    to_write = "insert into Emp_Emergency_Contact (Employee_ID, Emergency_Contact_Name, Relationship, Emergency_Contact_No, CREATED_BY) values ('" + str(row['ID']) + "', '" + row['Emergency Contact Name'] + "', '" + row['Relationship'] + "', '" + str(row['Emergency Contact Number ']) + "' , 1);\n"
    output_file.write(to_write)
    
    
output_file.close()

Extracting schedule information

In [4]:
schedule = pd.read_excel('K11 Employment Particulars and Roaster (Cleaned).xlsx', 'Feb 2020 (Adjusted)')
schedule.drop([0, 1, 31, 32, 33, 34, 35, 36], axis=0, inplace=True)
schedule.drop([schedule.columns[2], schedule.columns[3]], axis=1, inplace=True)
schedule
col_name = ['Site_ID', 'Location']
partial_date_shift = "2020-02-"
for i in range(1, 30):
    day = str(i)
    if len(day) == 1:
        day = '0' + day
    partial_date_shift += day + "|day" 
    col_name.append(partial_date_shift)
    partial_date_shift = "2020-02-"
    partial_date_shift += day + "|night" 
    col_name.append(partial_date_shift)
    partial_date_shift = "2020-02-"

schedule.columns = col_name
schedule

output_file = open("data extracted.sql","a")
unique_site = []
for index, row in schedule.iterrows():
    if row['Site_ID'] not in unique_site:
        unique_site.append(row['Site_ID'])
        # Site table
        to_write = "insert into site (Project_Name, Shifts, Public_Holiday, Site_Allowance, Employees_Required, Active, CREATED_BY, Address) values ('" + row['Location'] + "', '" + "day and night" + "', " + "true" + ", " + "0" + ", " + "1" + ", " + "true" + ", " + "1" + ", '" + row['Location'] + "');\n"
        output_file.write(to_write)
    
unique_site = []
for index, row in schedule.iterrows():
    for index2, row2 in info.iterrows():
        if row['Site_ID'] not in unique_site:
            unique_site.append(row['Site_ID'])
            # Schedule ranking table
            to_write = "insert into Schedule_Ranking (Employee_ID, Site_ID, Completed_Shift, CREATED_BY) values (" + str(row2['ID']) + ", " + str(row['Site_ID']) + ", " + "0" + ", 1);\n"
            output_file.write(to_write)

for index, row in schedule.iterrows():
    for date_shift in col_name:
        if date_shift != 'Location' and date_shift != 'Site_ID':
            date = date_shift.split('|')[0]
            year = date.split('-')[0]
            month = date.split('-')[1]
            day = date.split('-')[2]
            shift = date_shift.split('|')[1]
            
            person = row[date_shift]
      
            if type(person) == int:
                # Schedule table
                to_write = "insert IGNORE into schedule (`Year`, `Month`, `Day`, Site_ID, Shift, Employee_ID, `CREATED_BY`) values ('" + year + "', '" + month + "', '" + day + "', '" + str(row['Site_ID']) + "', '" + shift + "', '" + str(person) + "' , 1);\n"
                output_file.write(to_write)
                
                #Update schedule ranking table. Assumes shift is completed
                to_write = "update schedule_ranking set Completed_Shift = Completed_Shift + 1, LAST_MODIFIED_BY=1 where Employee_ID=" + str(person) + " and Site_ID=" + str(row['Site_ID']) + ";\n"
                output_file.write(to_write)
            else:
                # Schedule table
                to_write = "insert IGNORE into schedule (`Year`, `Month`, `Day`, Site_ID, Shift, Employee_ID, `CREATED_BY`) values ('" + year + "', '" + month + "', '" + day + "', '" + str(row['Site_ID']) + "', '" + shift + "', " + "1" + ", 1);\n"
                output_file.write(to_write)


output_file.close()